##### Last update - 15th April
Erik Hambardzumyan 

# Outline
### -  1. Data querying from Mimic IV
### -  2. Data processing

# 0. Basic imports & functions

In [2]:
import ast
import pickle
import psycopg2
import pandas as pd
pd.set_option('display.max_rows', 500)
import datetime
import numpy as np
from mlxtend.frequent_patterns import fpgrowth, apriori, fpmax
from mlxtend.preprocessing import TransactionEncoder
from spmf import Spmf
import shutup; shutup.please()

In [31]:
def execute_query_postgres(query):
    """
    Executes queries on postgresql database
    """
    
    # Establishing the connection
    conn = psycopg2.connect(
        dbname="mimiciv",
        user="",
        password="",
        host="localhost",
        port=5431
    )
    cursor = conn.cursor()
    
    # Executing a SQL query
    cursor.execute(query)
    
    record = cursor.fetchall()
    
    # Closing the cursor & connection
    cursor.close()
    conn.close()
    return record

def lab_and_pharma_events():
    query = """
    SELECT * FROM (
        SELECT SUBJECT_ID,
                HADM_ID,
                CHARTTIME,
                ITEMID::TEXT AS COL_ID,
                'labevents' AS TBL_NAME
                FROM MIMICIV_HOSP.LABEVENTS AS LABEVENTS_FULL
                UNION ALL 
        SELECT SUBJECT_ID,
                HADM_ID,
                COALESCE(VERIFIEDTIME,STARTTIME) AS CHARTTIME,
                MEDICATION AS COL_ID,
                'pharmacy' AS TBL_NAME
                FROM MIMICIV_HOSP.PHARMACY) as EVENTS
                WHERE EVENTS.HADM_ID is not NULL
            """
    records = execute_query_postgres(query)
    chunk_size = 100000  # ### Process data in batches, you can adjust this according to your system's capacity
    cols = ['subject_id', 'hadm_id', 'charttime', 'col_id', 'tbl_name']
    
    data_frames = []
    for i in range(0, len(records), chunk_size):
        chunk = records[i:i+chunk_size]
        df_chunk = pd.DataFrame(chunk, columns=cols)
        data_frames.append(df_chunk)
    
    # Union all chunks
    final_df = pd.concat(data_frames)
    final_df.to_csv("./full_db.csv", index=False)
    
def diagnosis_events():
    query = """Select 
    diag.subject_id, 
    diag.hadm_id,
    diag.seq_num,
    diag.icd_code,
    diag.icd_version,
    d_icd_diagnoses.long_title 
    from mimiciv_hosp.diagnoses_icd AS diag 
                JOIN mimiciv_hosp.d_icd_diagnoses AS d_icd_diagnoses ON 
    											diag.icd_code = d_icd_diagnoses.icd_code
    											AND diag.icd_version = d_icd_diagnoses.icd_version
    """
    
    records = execute_query_postgres(query)
    
    chunk_size = 100000  Process data in batches, you can adjust this according to your system's capacity
    cols = ['subject_id', 'hadm_id','seq_num', 'icd_code', 'icd_version', 'long_title']
    
    data_frames = []
    for i in range(0, len(records), chunk_size):
        chunk = records[i:i+chunk_size]
        df_chunk = pd.DataFrame(chunk, columns=cols)
        data_frames.append(df_chunk)
    
    # Union all chunks
    final_df = pd.concat(data_frames)
    final_df['icd_code'] = final_df['icd_code'].str.strip()
    final_df['icd_code'] = final_df['icd_code'].astype("str")+"_"+final_df['icd_version'].astype("str")
    final_df['subject_id'] = final_df['subject_id'].astype("str")+"_"+final_df['hadm_id'].astype("str")
    final_df.drop(columns="hadm_id", inplace=True)
    final_df["icd_code"] = "diag_"+final_df["icd_code"]
    final_df.to_csv("./diag3.csv", index=False)

def basic_preprocessing(df):
    """
    Drops nulls, combines subject_id and admission id, drops duplicates
    """
    df = df.dropna()
    df['charttime'] = pd.to_datetime(df['charttime'])
    df['hadm_id'] = df['hadm_id'].astype(int)
    # combine admission with subject_ids
    df['subject_id'] = df['subject_id'].astype("str")+"_"+df['hadm_id'].astype("str")
    df = df.drop_duplicates().drop(columns=["hadm_id"])
    return df
    
def map_mimic_to_loinc(df):
    """
    Maps lab events ids to loinc codes, the cases that aren't joined are filtered
    e.g. a lot of lab events are CBC (complete blood count)
    """
    lonic = pd.read_csv("./mimic_to_loinc.csv") 
    lonic.itemid = lonic.itemid.astype("string")
    joined_df = pd.merge(df.query("tbl_name=='labevents'"), lonic, how='inner', left_on='col_id', right_on='itemid')
    lab_events = joined_df[['subject_id','charttime', 'loinc_code', 'tbl_name']].dropna().drop_duplicates()
    lab_events = lab_events.rename(columns={"loinc_code":"col_id"})
    df = pd.concat([df.query("tbl_name!='labevents'"), lab_events]).reset_index(drop=True)
    return df

def convert_to_frozenset(loinc_list):
    return frozenset(loinc_list.split(', '))

def break_down(pattern, buckets, id_mapping_basket):
    """
    Breaking Down Same Time Concurrent Events (STCE). Algorithm 1 from
    Efficient Mining Template of Predictive Temporal Clinical Event Patterns From Patient Electronic Medical Records
    by Jianqiang Li et al.

    The idea is to find longest subset by length that maps to a group, then the processes is repeated for remaining items.
    For instance, if a patient orders lab items "ABCDE," and "ABC" belong to a panel or frequent basket, they are assigned basket code,
    and the algorithm iterates on the remaining "DE".
    """
    best_subsets = []
    output = pattern 
    for B in buckets:
        if len(B[0])<len(pattern):
            for SE in B:
                if SE.issubset(pattern):
                    best_subsets.append(SE)
                    pattern = pattern.difference(SE)
                    if not pattern:
                         break
    if len(best_subsets)>0:
        best_subsets = [id_mapping_basket[best_subset] if len(best_subset) > 1 else best_subset for best_subset in best_subsets]
        if pattern:
            best_subsets.append(pattern)
        output = frozenset.union(*best_subsets)
    return list(output)

def convert_loinc_to_panel(df):
    """
    Group loinc ids into panels
    """
    loinc_panels = pd.read_csv("./mimic_loinc_panels.csv")
    # convet lists to frozen sets and add length of LoincList
    loinc_panels['LoincList'] = loinc_panels['LoincList'].apply(convert_to_frozenset) 
    loinc_panels['ParentLoinc'] = loinc_panels['ParentLoinc'].apply(lambda x: frozenset([x]))
    loinc_panels['length'] = loinc_panels['LoincList'].apply(len)
    # group panels by length
    loinc_dict = dict(zip(loinc_panels['LoincList'], loinc_panels['ParentLoinc']))
    buckets_loinc = (loinc_panels.groupby(['length'])['LoincList']
                                 .apply(list).reset_index(name='buckets')
                                 .sort_values("length",ascending=False)['buckets'].to_list()) 
    
    lab_panels = group_into_list(df.query("tbl_name=='labevents'"))
    # this is where magic happens
    lab_panels['col_id_list'] = lab_panels["col_id_list"].apply(lambda x: break_down(frozenset(x), buckets_loinc, loinc_dict))
    # after we have them in a list, we have to explode the table back to its original form
    lab_panels = lab_panels.explode('col_id_list').rename(columns={'col_id_list': 'col_id'})
    lab_panels['tbl_name'] = 'labevents'
    # add other tables  
    df = pd.concat([df.query("tbl_name!='labevents'"), lab_panels]).reset_index(drop=True)
    return df
    
def generate_string_pharm(df):
    """
    Generate unique 5-length digit IDs for each pharmacy strings
    """
    unique_strings = df.query("tbl_name=='pharmacy'")['col_id'].drop_duplicates().tolist()
    id_mapping = {string: f"{index:05}" for index, string in enumerate(unique_strings)}
    pharm_mapping = pd.DataFrame(list(id_mapping.items()), columns=['col_id', 'new_col_id'])
    return pharm_mapping

def map_pharmacy_codes(df, pharm_mapping):
    """
    Map pharm names to 5-length digits
    """
    df = pd.merge(df, pharm_mapping, how='left', left_on='col_id', right_on='col_id')
    df['col_id'] = (np.where(df['tbl_name'] == 'pharmacy', df['new_col_id'], df['col_id']))
    df = df.drop(columns=["new_col_id"])
    return df

def add_prefix(df):
    """
    Add a prefix based on 3 characters of each tbl_name to col_id
    """
    # create a prefix based on 3 characters of each tbl_name
    df['prefix'] = df['tbl_name'].apply(lambda x: x[0:3]+'_')
    # add the prefix to the col_id
    df['col_id'] = df['prefix']+df['col_id']
    df = df.drop(columns=["prefix"])
    return df

def group_into_list(df):
    """
    For each subject_id obtain list of all items that were charted together
    """
    df = df.sort_values(['subject_id','charttime'], ascending=[True, True])
    # add all items to a list 
    df = (df.groupby(['subject_id', 'charttime'])['col_id'].apply(list)
            .reset_index(name='col_id_list'))
    return df
    
def generate_frequent_itemsets(df, min_support):
    """
    Generate frequent itemsets
    """
    te = TransactionEncoder()
    te_ary = te.fit(df[ "col_id_list"].to_list()).transform(df[ "col_id_list"].to_list())
    onehot = pd.DataFrame(te_ary, columns=te.columns_)
    patterns = fpmax(onehot, min_support=min_support, use_colnames=True)
    patterns['itemsets'] = patterns['itemsets'].apply(lambda x: frozenset(x))
    return patterns

def get_buckets(patterns):
    """
    Buckets aim to group items by by descending length and within each bucket, items are sorted based on their support (highest first)
    """
    patterns['length'] = patterns['itemsets'].apply(len)
    patterns = patterns.sort_values(["length", "support"],ascending=[False, False])
    buckets = (patterns
               .groupby(['length'])['itemsets'].apply(list).reset_index(name='buckets')
              ).sort_values("length",ascending=False)['buckets'].to_list()       
    
    return buckets

def basket_mappings(patterns):
    """
    Create unique IDs for frequent baskets 
    """
    long_baskets = patterns.query("length>1")['itemsets'].drop_duplicates().to_list()
    id_mapping_basket = {string: frozenset({f"basket_{index:05}"}) for index, string in enumerate(long_baskets)}
    mapping_basket_df = pd.DataFrame(list(id_mapping_basket.items()), columns=['basket', 'basket_id'])
    return id_mapping_basket, mapping_basket_df

def convert_string_list(string_list):
    return [list(map(int, s.split())) if ' ' in s else [int(s)] for s in string_list]


def to_pandas_dataframe(self, pickle=False):
    """
    Convert output to pandas DataFrame
    pickle: Save as serialized pickle
    """
    # TODO: Optional parameter for pickle file name

    if not self.patterns_:
        self.parse_output()

    patterns_dict_list = []
    for pattern_sup in self.patterns_:
        pattern = pattern_sup[:-1]
        sup_info = pattern_sup[-1:][0]
        sup = int(sup_info.split("#SUP:")[1].split()[0])
        sids = [int(sid) for sid in sup_info.split("#SID:")[1].strip().split()]

        patterns_dict_list.append({'pattern': pattern, 'sup': sup, 'sid_list': sids})
    
    df = pd.DataFrame(patterns_dict_list)
    self.df_ = df

    if pickle:
        df.to_pickle(self.output_.replace(".txt", ".pkl"))
    return df

def combine_datasets():
    df = pd.read_csv("./ready_for_seq_mining.csv", converters={'col_id_list': ast.literal_eval})
    df['len'] = df['col_id_list'].apply(len)
    df['eventID'] = (df
        .sort_values(['subject_id','charttime'], ascending=[True, True])
        .groupby(['subject_id']).cumcount() + 1)
    # make sure subject and seq num is unique otherwise it's a bug
    diag = pd.read_csv("diag3.csv").drop_duplicates(subset=['subject_id', 'seq_num'])
    
    # Take maximum seq number for each subject
    max_event = diag.groupby('subject_id')['seq_num'].max().reset_index(name='seq_num')
    
    # Join it with main events and make sure that eventID is after the maximum seq number of the diagnosis
    df = pd.merge(df,max_event, on='subject_id')
    df["eventID"] = df["eventID"]+df["seq_num"]
    
    # rename
    diag.rename(columns=dict(zip(diag.columns, ["subject_id","eventID","col_id_list"])), inplace=True)
    diag['col_id_list'] = diag['col_id_list'].apply(lambda x:[x])
    # only keep those diagnosis events that have lab events
    diag = pd.merge(df[['subject_id']].drop_duplicates(), diag, on='subject_id')
    
    # Add diagnoses
    unioned = pd.concat([df[['subject_id',"eventID", "col_id_list"]], 
                       diag[['subject_id',"eventID", "col_id_list"]]])
    # add lengths of itemsets
    unioned['length_items'] = unioned['col_id_list'].apply(len)
    
    # sort by "subject_id", "eventID"
    unioned = (unioned[['subject_id', "eventID", "length_items","col_id_list"]]
              .sort_values(["subject_id", "eventID"])
    ).reset_index(drop=True)

    unioned.to_csv('arules_input_final_mimic_raw.txt', index=False)

# 1. Data querying from Mimic IV
[Mimic-IV](https://mimic.mit.edu/docs/iv/) contains real data of patients’ hospital admissions to a tertiary academic medical center in Boston, MA, USA (also see [github repository](https://github.com/MIT-LCP/mimic-code/tree/main/mimic-iv)). Our primary focus lies on the 'hosp' (hospital) module, housing comprehensive data such as laboratory orders and prescribed medications. It's important to note that not all data within this module originates directly from the hospital; some may stem from the emergency department, albeit stored within the hospital's Electronic Health Record (EHR) system.

For our data mining purposes, we combine two tables: 'labevents' for laboratory measurements and 'pharmacy' for prescribed medications. The selected columns include:

- SUBJECT_ID - identifier of the patient.
- HADM_ID - identifier of the admission.
- CHARTTIME - The time at which the laboratory measurement was charted.,
for pharmacy it is the time that the prescription was verified or entered into the system.
- COL_ID - An identifier which uniquely denotes laboratory items, or presciption name for pharmacy.
- TBL_NAME - signififies the table source 'pharmacy' or 'labevents'

We are interesed in data within one admission, therefore we filter out cases where *hadm_id* is absent. hile there are a total of 431K admissions, not all lab events are linked to an 'hadm_id'. The absence of this identifier suggests that the lab sample may have been collected outside the hospital premises. Outside the hospital encompasses areas such as the emergency department, which operates as a clinic until the patient is formally admitted (see the [discussion thread](https://github.com/MIT-LCP/mimic-code/issues/1703)). Consequently, only 351K admissions are associated with a lab event. 


To elaborate on the nature of lab events, specimens are collected from a patient, such as a urine sample to measure pH or a blood sample for a complete blood count. The 'Charttime' timestamp denotes when the specimen was collected. Consequently, multiple orders may share the same specimen charttime because a single sample can be used for different laboratory measurements.

#### 1.1 Diagnosis events extraction

In [80]:
diagnosis_events()

In [5]:
pd.read_csv("./diag3.csv").head(5)

,subject_id,seq_num,icd_code,icd_version,long_title
0,10000032_22595853,2,diag_78959_9,9,Other ascites
1,10000032_22595853,4,diag_07070_9,9,Unspecified viral hepatitis C without hepatic ...
2,10000032_22595853,6,diag_29680_9,9,"Bipolar disorder, unspecified"
3,10000032_22595853,7,diag_30981_9,9,Posttraumatic stress disorder
4,10000032_22841357,2,diag_78959_9,9,Other ascites


#### 1.2 Obtain pharmacy and lab events raw data 

In [ ]:
lab_and_pharma_events()

In [7]:
pd.read_csv("./full_db.csv").head(5)

,subject_id,hadm_id,charttime,col_id,tbl_name
0,12968666,26897334,2116-12-29 10:15:00,50912,labevents
1,12967791,27191750,2195-08-27 10:15:00,50856,labevents
2,12967791,27191750,2195-08-27 10:15:00,50868,labevents
3,12967791,27191750,2195-08-27 10:15:00,50879,labevents
4,12967791,27191750,2195-08-27 10:15:00,50880,labevents


#### Obtain all Mimic IV admissions

In [14]:
query = """Select count(*) from MIMICIV_HOSP.admissions"""

execute_query_postgres(query)

[(431231,)]

# 2. Data preprocessing
Here is the outline of steps:
-  2.1 Basic preprocessing
-  2.2 Map item ids to loinc codes
-  2.3 Group loinc codes to panels
-  2.4 Map 5-digit codes to pharmacy
-  2.5 Filter out subjects who had presriptions only and no labevents
-  2.6 For each subject create an ordered list of item orders to prepare for data mining
-  2.7 STCE breakdown with frequent baskets (a.k.a itemsets)
- 2.8 Combine datasets

In [44]:
df = pd.read_csv("./full_db.csv").query("tbl_name!='microbiologyevents'").query("tbl_name!='procedures_icd'")

##### How many concurrent events per event type?

In [51]:
df.groupby(["subject_id", "hadm_id", "charttime", "tbl_name"]).size() \
  .reset_index(name='size') \
  .groupby("tbl_name") \
  .agg({"size": np.mean})

,size
tbl_name,
labevents,16.757969
pharmacy,1.941792


#### 2.0 Raw table: labevents 60.6M labevents 

In [20]:
## transactions per table
print(df.groupby("tbl_name").size())
# admissions per table
print(df[['hadm_id', 'tbl_name']].drop_duplicates().groupby("tbl_name").size())

tbl_name
labevents    60601911
pharmacy     12689767
dtype: int64
tbl_name
labevents    351034
pharmacy     365405
dtype: int64


#### 2.1 Basic preprocessing
This step includes combining subject id with admission id (*hadm_d*), filtering "Sodium Chloride 0.9%  Flush" from pharmacy,
      finally removing any null values or duplicate rows.

##### How many patients were presribed "Sodium Chloride 0.9%  Flush"?

In [13]:
(df.query("col_id == 'Sodium Chloride 0.9%  Flush'")['subject_id'].drop_duplicates().shape[0]/
df['subject_id'].drop_duplicates().shape[0)]*100

91.36015691983148

In [23]:
# very repetitive pharmacy, needs to be removed
df = df.query("col_id != 'Sodium Chloride 0.9%  Flush'")
df = basic_preprocessing(df)

In [28]:
df.head(3)

,subject_id,charttime,col_id,tbl_name
0,12968666_26897334,2116-12-29 10:15:00,50912,labevents
1,12967791_27191750,2195-08-27 10:15:00,50856,labevents
2,12967791_27191750,2195-08-27 10:15:00,50868,labevents


##### 60.6M to 60.5M after basic preprocessing

In [27]:
print(df.groupby("tbl_name").size())
print(df[['subject_id', 'tbl_name']].drop_duplicates().groupby("tbl_name").size())

tbl_name
labevents    60495669
pharmacy     10862892
dtype: int64
tbl_name
labevents    351034
pharmacy     365269
dtype: int64


#### 2.2 Map item ids to loinc codes
Logical Observation Identifiers Names and Codes (LOINC) is a database which aims to standardize medical laboratory test orders

In [29]:
df.col_id = df.col_id.astype("string")
df = map_mimic_to_loinc(df)

In [36]:
df.query("tbl_name=='labevents'").head(3).reset_index(drop=True)

,subject_id,charttime,col_id,tbl_name
0,12968666_26897334,2116-12-29 10:15:00,2160-0,labevents
1,12967791_27191750,2195-08-27 10:15:00,3298-7,labevents
2,12967791_27191750,2195-08-27 10:15:00,3376-1,labevents


##### After joining with loinc codes 60.5M labevents is reduced to 36.9M

In [37]:
print(df.groupby("tbl_name").size())
print(df[['subject_id', 'tbl_name']].drop_duplicates().groupby("tbl_name").size())

tbl_name
labevents    36922975
pharmacy     10862892
dtype: int64
tbl_name
labevents    350904
pharmacy     365269
dtype: int64


In [34]:
#df.to_csv("./full_db_basic_preprocessed.csv", index=False)

#### 2.3 Group loinc codes to panels 
[Loinc Panels](https://loinc.org/51992-6/panel) are groups of loinc codes (identified by parent id) of related laborotary items. Since a lot lab items are ordered concurrently, it is meaningul to group them to reduce both computaitonal complexity and intepretability of results. For example, loinc parent id *51992-6* refers to heavy meatals panel contains items such as Copper or Zinc test ([Mass/volume] in Serum or Plasma). We implement an algorithm for mapping Loinc codes to parents based on the method introduced by Jianqiang Li et al. ([link](https://pubmed.ncbi.nlm.nih.gov/30346297/)), known as Breaking Down Same Time Concurrent Events (STCE). The concept involves identifying the largest subset in the sequence of ordered lab tests that corresponds to a panel. For instance, if a patient orders lab items "ABCDE," and "ABC" belong to a panel, they are assigned the panel code, and the algorithm iterates on the remaining "DE". Any items that aren't mapped to a panel code remain unchanged.

In [18]:
df = pd.read_csv("./full_db_basic_preprocessed.csv")

In [19]:
df = convert_loinc_to_panel(df)

##### After groupping to panels 36.9M is reduced to 28.1M

In [45]:
print(df.groupby("tbl_name").size())
print(df[['subject_id', 'tbl_name']].drop_duplicates().groupby("tbl_name").size())

tbl_name
labevents    28099290
pharmacy     10862892
dtype: int64
tbl_name
labevents    350904
pharmacy     365269
dtype: int64


#### 2.4 Map 5-digit codes to pharmacy
Since medications in MIMIC lack associated IDs, we assign them unique 5-digit codes for enumeration. To indicate the origin of each item ID, we prepend the first three letters of each table as prefixes to the codes ('lab' for labevents, 'pha' for pharmacy).

In [21]:
pharm_mapping = generate_string_pharm(df)
df = map_pharmacy_codes(df, pharm_mapping)
df = add_prefix(df)

In [42]:
pharm_mapping.to_csv("./pharm_mapping.csv", index=False)

In [6]:
df.head(3)

,subject_id,charttime,col_id,tbl_name
0,16552715_25857688,2142-06-15 10:00:57,pha_00000,pharmacy
1,16552715_25857688,2142-06-14 16:39:12,pha_00001,pharmacy
2,16552715_25857688,2142-06-15 09:36:45,pha_00002,pharmacy


In [ ]:
#df.to_csv("./full_db_basic_preprocessed_after_prefix.csv", index=False)

In [16]:
df = pd.read_csv("./full_db_basic_preprocessed_after_prefix.csv")

#### 2.5 Filter out subjects who had presriptions only and no labevents
We only care about those subjects who had both labevents and prescriptions, those who only had prescriptions are filtered out

##### Pharmacy transactions are reduced from 10.9M to 105.6M. 

In [15]:
df = pd.merge(df, df.query("tbl_name=='labevents'")[['subject_id']].drop_duplicates(), on='subject_id')

In [16]:
print(df.groupby("tbl_name").size())
print(df[['subject_id', 'tbl_name']].drop_duplicates().groupby("tbl_name").size())

tbl_name
labevents    28099290
pharmacy     10558571
dtype: int64
tbl_name
labevents    350904
pharmacy     340034
dtype: int64


#### 2.6 For each subject create an ordered list of item orders to prepare for itemset mining

In [68]:
df = group_into_list(df)

In [53]:
df.head(5)

,subject_id,charttime,col_id_list
0,10000032_22595853,2180-05-07 00:10:00,"[lab_5792-7, lab_3397-7, lab_5802-4, lab_5811-..."
1,10000032_22595853,2180-05-07 05:05:00,"[lab_6768-6, lab_50676-6, lab_CBC, lab_1742-6,..."
2,10000032_22595853,2180-05-07 10:11:00,"[lab_2531-2, lab_30380-0, lab_26488-7, lab_519..."
3,10000032_22841357,2180-06-26 22:45:00,"[lab_5792-7, lab_5822-2, lab_5804-0, lab_8247-..."
4,10000032_22841357,2180-06-27 05:10:00,"[lab_6768-6, lab_1742-6, lab_CBC, lab_1975-2, ..."


In [96]:
df.to_csv("./ready_for_breakdown.csv", index=False)

#### 2.7 STCE breakdown with frequent baskets (a.k.a itemsets):
In this phase, our goal is to further combine lab events that frequently occur together. While we've already grouped some items by panels, leveraging hospital data allows us to identify commonly co-occurring items. Initially, we generate Frequent Item Baskets using the FPmax ([link](https://www.philippe-fournier-viger.com/spmf/fpmax.pdf)) algorithm, with user-defined minimum support criteria. For instance, if (lab_2075-0, lab_CBC) has a support of 45%, it indicates that 45% of all lab orders included both items, regardless of order. Additionally, we post-process frequent baskets, ensuring that only closed frequent baskets remain, meaning that no superset of the basket can be frequent.

Subsequently, each basket is assigned a unique ID. Baskets are then grouped into "buckets" by length descending, and within each bucket, items are sorted based on their support (highest first). Finally, we reapply the STCE breakdown algorithm, but instead of using parent LOINCs, we utilize the frequent baskets.

In [4]:
df = pd.read_csv("./ready_for_breakdown.csv", converters={'col_id_list': ast.literal_eval})
filtered_df = df[df['col_id_list'].apply(lambda x: len(x) > 1)]

##### Number of max frequent baskets with differnet support levels

In [24]:
supports = [0.5, 0.4, 0.3, 0.2, 0.1, 0.05]
for min_sup in supports:
    patterns = generate_frequent_itemsets(filtered_df, min_sup)
    print("MinSup:{s}".format(s=min_sup), "Number of patterns:{n}".format(n=patterns.shape[0]))

MinSup:0.5 Number of patterns:0
MinSup:0.4 Number of patterns:2
MinSup:0.3 Number of patterns:3
MinSup:0.2 Number of patterns:1
MinSup:0.1 Number of patterns:18
MinSup:0.05 Number of patterns:20


In [8]:
# It seems? aorund 10% is optimal because lower supports yields similar number of patterns
patterns = generate_frequent_itemsets(filtered_df, 0.10)
# Add lenth of each basket (a.k.a. itemset)
patterns['length'] = patterns['itemsets'].apply(len)

In [10]:
pd.set_option('display.max_colwidth', None)
patterns

,support,itemsets,length
0,0.100637,"(lab_2075-0, lab_34548-8, lab_1963-8, lab_1975-2, lab_CBC)",5
1,0.100018,"(lab_34548-8, lab_1963-8, lab_1975-2, lab_1920-8, lab_6768-6)",5
2,0.100043,"(lab_2075-0, lab_1963-8, lab_1975-2, lab_1920-8, lab_6768-6)",5
3,0.100005,"(lab_2075-0, lab_34548-8, lab_1963-8, lab_6768-6, lab_1975-2, lab_1742-6)",6
4,0.101068,"(lab_2075-0, lab_34548-8, lab_1963-8, lab_1975-2, lab_1920-8, lab_1742-6)",6
5,0.100140,"(lab_2075-0, lab_34548-8, lab_6768-6, lab_1975-2, lab_1920-8, lab_1742-6)",6
6,0.101232,"(lab_2075-0, lab_34548-8, lab_1963-8, lab_6768-6, lab_1920-8, lab_CBC, lab_1742-6)",7
8,0.100005,"(lab_14979-9, lab_2075-0, lab_70219-1, lab_19123-9)",4
9,0.101886,"(lab_2075-0, lab_34548-8, lab_1963-8, lab_5902-2, lab_14979-9, lab_70219-1, lab_6301-6, lab_CBC)",8
10,0.124893,"(lab_2075-0, lab_19123-9, lab_34548-8, lab_1963-8, lab_5902-2, lab_14979-9, lab_6301-6, lab_CBC)",8


##### Apply the STCE algorithm

In [32]:
# Obtain unique ids for each basket
id_mapping_basket, mapping_basket_df = basket_mappings(patterns)
buckets = get_buckets(patterns)
# Perform the STCE breakdown algorithm
df['col_id_list'] = df["col_id_list"].apply(lambda x: break_down(frozenset(x), buckets, id_mapping_basket))

In [ ]:
mapping_basket_df.to_csv("./mapping_basket_df.csv", index=False)

##### After STCE breakdown (with 10% min sup), labevents are reduced to 20.5M from 28.1M

In [35]:
temp = df.explode('col_id_list')
pha_count = temp[temp['col_id_list'].str.startswith('pha')].shape[0]
lab_count = temp[temp['col_id_list'].str.startswith('lab')].shape[0]
print("Number of rows with 'pha':", pha_count)
print("Number of rows with 'lab':", lab_count)

Number of rows with 'pha': 10558571
Number of rows with 'lab': 20487101


In [37]:
# df.to_csv("./ready_for_seq_mining.csv", index=False)

-----

####  2.8 Combine datasets

In [ ]:
combine_datasets()

##### How many diagnosis events before combining?

In [23]:
pd.read_csv("diag3.csv").drop_duplicates(subset=['subject_id', 'seq_num']).shape[0]

4756210

##### Counts after preprocessing

In [25]:
temp = pd.read_csv("arules_input_final_mimic_raw.txt", converters={'col_id_list': ast.literal_eval}).explode('col_id_list')
pha_count = temp[temp['col_id_list'].str.startswith('pha')].shape[0]
lab_count = temp[temp['col_id_list'].str.startswith('lab')].shape[0]
diag_count = temp[temp['col_id_list'].str.startswith('diag')].shape[0]

In [28]:
print("pharmacy:", pha_count)
print("labevents:", lab_count)
print("diagnoses:", diag_count)

pharmacy: 9613364
labevents: 15045636
diagnoses: 4313675


In [8]:
### aruleSequences input
def list_to_str(lst):
    return '\t'.join(map(str, lst)) if isinstance(lst, list) else ''
unioned['col_id_list'] = unioned['col_id_list'].apply(list_to_str)

In [10]:
# write to txt
unioned.to_csv('arules_input_final.txt', 
                 sep=' ', 
                 index=False, 
                 header=False,
                 na_rep='')

---

## Mappings behind codes

In [2]:
loinc_panels = pd.read_csv("./mimic_loinc_panels.csv")
lonic = pd.read_csv("./mimic_to_loinc.csv") 
pharm_mapping = pd.read_csv("./pharm_mapping.csv", dtype={'new_col_id': str})
mapping_basket_df = pd.read_csv("./mappings/mapping_basket_df.csv", converters={'basket': eval,'basket_id': eval})